In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from array import *
import numpy as np

def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath,lines = True)

    # insert song record
    df_songs = df[['song_id', 'title','artist_id','year','duration']]
    df_songs = df_songs.dropna()
    df_songs = df_songs.drop_duplicates(subset='song_id', keep="last", inplace=False)
    arr_songs = df_songs.values
    song_data = np.array(arr_songs).tolist()
    cur.execute(song_table_insert, song_data[0])
    
    # insert artist record
    df_artists = df[['artist_id', 'artist_name','artist_location','artist_latitude','artist_longitude']]
    df_artists = df_artists.dropna()
    df_artists = df_artists.drop_duplicates(subset='artist_id', keep="last", inplace=False)
    arr_artists = df_artists.values
    artist_data = np.array(arr_artists).tolist()
    if (len(artist_data) > 0):
        cur.execute(artist_table_insert, artist_data[0])


def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath,lines = True)
    df = df.dropna()

    # filter by NextSong action
    df = df[(df.page == 'NextSong')]

    # convert timestamp column to datetime
    t = pd.to_datetime(df['ts'], unit='ms')
    
    # insert time data records
    time_data = [t.dt.time,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday]
    column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    t_dict = dict(zip(column_labels,time_data))
    time_df = pd.DataFrame.from_dict(t_dict)
    time_df = time_df.dropna()
    time_df = time_df.drop_duplicates(subset='start_time', keep="last", inplace=False)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
    user_df = user_df.dropna()
    user_df = user_df.drop_duplicates(subset='userId', keep="last", inplace=False)

    # insert user records
    for i, row in user_df.iterrows():
            cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        t1 = pd.to_datetime(row.ts, unit='ms')
   
        songplay_data = (t1, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)


def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))
        print(datafile)


def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()